In [1]:
from robotic import ry
import numpy as np
import time

###########################################PART-1 STARTS HERE##########################################

In [7]:
C = ry.Config()
C.addFile('cargobot_base.g')
C.addFile('maze.g')

qHome = C.getJointState()
C.setJointState(qHome)

In [ ]:
#Adding waypoints


Get Final Pose with KOMO

In [8]:
komo = ry.KOMO(C, 1, 1, 2, True)
komo.addControlObjective([], 0, 1e-1)
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e2])
komo.addObjective([1], ry.FS.positionDiff, ['base', 'goal_area'], ry.OT.eq, [1e2])

ret = ry.NLP_Solver(komo.nlp(), verbose=0).solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)

qT = komo.getPath()[0]
C.setJointState(qT)
C.view(False, "IK solution")


{ time: 0.004575, evals: 52, done: 1, feasible: 0, sos: 29.5425, f: 0, ineq: 0, eq: 5.5 }
size of path: (1, 3)


0

RRT

In [9]:
#define a path finding problem
rrt = ry.PathFinder()
i = 0
while True:
    i = i + 1
    print(i)
    rrt.setProblem(C, [qHome], [qT])
    ret = rrt.solve()
    path = ret.x
    print("rettttt: ", ret)
    #print("path size:", path)
    # display the path
    if hasattr(ret, 'x') and isinstance(ret.x, np.ndarray) and ret.x.ndim == 2 and ret.x.shape[0] > 0:
        for t in range(0, path.shape[0] - 1):
            print("üffff: ", path[t])
            C.setJointState(path[t])
            C.view()
            time.sleep(0.002)
        break

1
rettttt:  { time: 0.335065, evals: 5001, done: 1, feasible: 0, sos: -1, f: -1, ineq: -1, eq: -1 }
2
rettttt:  { time: 0.710516, evals: 10002, done: 1, feasible: 0, sos: -1, f: -1, ineq: -1, eq: -1 }
3
rettttt:  { time: 1.0954, evals: 15003, done: 1, feasible: 0, sos: -1, f: -1, ineq: -1, eq: -1 }
4
rettttt:  { time: 1.42093, evals: 19726, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }
üffff:  [0. 0. 0.]
üffff:  [-0.04052937  0.02720767  0.01082187]
üffff:  [-0.08189819 -0.00087459  0.01073812]
üffff:  [-0.12326702 -0.02895686  0.01065437]
üffff:  [-0.16477706 -0.05683001  0.01071435]
üffff:  [-0.20614451 -0.08491429  0.01062921]
üffff:  [-0.24751198 -0.11299854  0.01054409]
üffff:  [-0.28887945 -0.14108279  0.01045897]
üffff:  [-0.28960577 -0.19044129  0.01840953]
üffff:  [-0.27065044 -0.23482197  0.03148815]
üffff:  [-0.31262795 -0.26198198  0.03101623]
üffff:  [-0.35485082 -0.28876302  0.03094617]
üffff:  [-0.34348206 -0.33742881  0.0293992 ]
üffff:  [-0.38563232 -0.36432

Trajectory Optimization via KOMO

In [5]:
C.setJointState(qHome)
komo2 = ry.KOMO(C, 1, path.shape[0], 2, True)
komo2.addControlObjective([], 0, 1e-1)
komo2.addControlObjective([], 2, 1e0)
komo2.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e2])
komo2.addObjective([1], ry.FS.positionDiff, ['base', 'goal_area'], ry.OT.eq, [1e2])
komo2.initWithPath_qOrg(path)

ret = ry.NLP_Solver(komo2.nlp(), verbose=0).solve()
q = komo2.getPath()
print('last path:', q)

# display the path for part 1
for i in range(0, q.shape[0]):
    C.setJointState(q[i])
    C.view(False, f'waypoints{i}')
    time.sleep(0.02)


last path: [[ 3.56821785e-05 -7.26208863e-04 -4.48713801e-06]
 [ 1.09649042e-04 -2.16855756e-03 -1.34446508e-05]
 [ 2.24503096e-04 -4.31697705e-03 -2.68557751e-05]
 ...
 [-4.12301230e+00 -3.20683646e+00  6.60080594e-01]
 [-4.15873110e+00 -3.22874231e+00  6.63031947e-01]
 [-4.19446114e+00 -3.25063894e+00  6.65983299e-01]]


###########################################PART-1 ENDS HERE##########################################